***Vorlesung 'Syntax natürlicher Sprachen'***

---
# PCFG: Abschätzung von Regelwahrscheinlichkeiten aus CFG-Korpusdaten

In [1]:
import nltk

In [2]:
from collections import defaultdict
from nltk.corpus import treebank

In [3]:
def find_relevant_constructions(lhs, only_with=None):
    lhs_nt = nltk.grammar.Nonterminal(lhs)
    should_filter = only_with is not None
    if should_filter:
        filter_by = list(map(nltk.grammar.Nonterminal, only_with))
        def passes_filter(tup):
            for f in filter_by:
                if f not in tup:
                    return False
            return True

    counter = defaultdict(int)
    ### zähle Produktionen in treebank mit lhs als linker Seite ###
    ### und einer rechten Seite, für die passes_filter True liefert ###
    for tree in treebank.parsed_sents():  # [(S (NP DET N) (VP V NP))]
        for prod in tree.productions():   # [(S, NP VP), (NP, DET N)]
            if prod.lhs() == lhs_nt:
                if not should_filter or passes_filter(prod.rhs()):
                    counter[prod] += 1

    return [ (k, counter[k]) for k in sorted(counter.keys(), key=counter.__getitem__) ]

---
## 1: Herunterladen von Ressourcen

#### Laden Sie sich zunächst die Ressource `corpora/treebank` über den NLTK Download-Manager herunter.

In [4]:
# nltk.download()

---
## 2: Von Daten zu Regelwahrscheinlichkeiten

#### Gegeben sei folgende kontextfreie Grammatik:

In [5]:
grammar = nltk.CFG.fromstring("""
S -> NP VP
VP -> V NP PP
VP -> V NP
NP -> DET N
NP -> NP PP
PP -> P NP

DET -> "the" | "a"
N -> "boy" | "woman" | "telescope"
V -> "saw"
P -> "with"
""")

#### Sie modelliert sehr einfache Sätze der Form `SBJ` *saw* `OBJ` mit optionaler Präpositionalphrase am Ende. Diese Präpositionalphrase kann entweder der näheren Bestimmung des Objekts oder der näheren Bestimmung der in der Verbalphrase ausgedrückten Handlung dienen.

In [6]:
sentence = "the boy saw a woman with a telescope"
parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)

                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌───────┬────┴─────────┐                
     │       │       │              PP              
     │       │       │         ┌────┴───┐            
     NP      │       NP        │        NP          
 ┌───┴───┐   │   ┌───┴────┐    │    ┌───┴──────┐     
DET      N   V  DET       N    P   DET         N    
 │       │   │   │        │    │    │          │     
the     boy saw  a      woman with  a      telescope

                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌────────────┴────┐                     
     │       │                 NP                   
     │       │       ┌─────────┴────┐                
     │       │       │              PP              
     │       │       │         ┌────┴

---
 
#### Im folgenden sollen aus der Penn Treebank Wahrscheinlichkeiten für die einzelnen Regeln extrahiert werden, um dieser Ambiguität Herr zu werden.

#### Nutzen Sie das im NLTK enthaltene Sample der Penn Treebank (nach Installation unter `nltk.corpus.treebank` zu finden) zunächst zur Identifikation der für eine Disambiguierung nützlichen (Teil-)bäume der Penn Treebank.

#### *Hinweis:* Sie können sich bei der Analyse auf die häufigsten Konstruktionen der Baumbank beschränken.

---
### 2.1: Regelwahrscheinlichkeiten für VP-Regeln abschätzen



---

#### Zählen Sie zunächst für die V+NP+PP-Konstruktion, wie oft sie in der Penn Treebank vorkommen und berechnen Sie die relativen Häufigkeiten als Approximation der Regelwahrscheinlichkeiten:

$$P(V, N\!P, P\!P \mid V\!P) = \dfrac{count(V\!P \rightarrow V\:N\!P\:P\!P)}{count(V\!P \rightarrow \setminus*)}$$

**hier (vgl. Grammatik):**

$$= \dfrac{count(V\!P \rightarrow V\:N\!P\:P\!P)}{count(V\!P \rightarrow V\:N\!P\:P\!P) + count(V\!P \rightarrow V\:N\!P)}$$

In [7]:
constructions = find_relevant_constructions('VP')
constructions[-20:]

[(VP -> VBZ NP-PRD, 163),
 (VP -> VBN NP PP, 170),
 (VP -> VBN NP PP-CLR, 178),
 (VP -> VBP NP, 185),
 (VP -> VBZ SBAR, 197),
 (VP -> VBZ S, 215),
 (VP -> VBD S, 223),
 (VP -> VP CC VP, 234),
 (VP -> VBN NP, 250),
 (VP -> VB VP, 258),
 (VP -> VBZ NP, 261),
 (VP -> VBP VP, 337),
 (VP -> VBD VP, 361),
 (VP -> VBG NP, 375),
 (VP -> VBD NP, 378),
 (VP -> VBZ VP, 459),
 (VP -> VBD SBAR, 631),
 (VP -> MD VP, 759),
 (VP -> VB NP, 805),
 (VP -> TO VP, 1257)]

In [8]:
vp_v_np_pp_frq = 178 + 170
vp_v_np_without_frq = 805 + 378 + 375 + 261 + 250 + 185 + 163

vp_with_pp = vp_v_np_pp_frq / (vp_v_np_pp_frq + vp_v_np_without_frq)
vp_without = vp_v_np_without_frq / (vp_v_np_pp_frq + vp_v_np_without_frq)

(vp_with_pp, vp_without)

(0.12585895117540688, 0.8741410488245931)

---
### 2.2: Regelwahrscheinlichkeiten für NP-Regeln abschätzen

#### Zählen Sie anschließend, wie oft die NP+PP-Konstruktion in der Penn Treebank vorkommt und berechnen Sie die relativen Häufigkeiten als Approximation der Regelwahrscheinlichkeiten. Das Vorgehen wird in folgender Formel veranschaulicht:

$$P(N\!P, P\!P \mid N\!P) = \dfrac{count(N\!P \rightarrow \:N\!P\:P\!P)}{count(N\!P \rightarrow \setminus*)}$$

**hier:**

$$= \dfrac{count(N\!P \rightarrow \:N\!P\:P\!P)}{count(N\!P \rightarrow \:N\!P\:P\!P) + count(N\!P \rightarrow DET\:N\!P)}$$



In [9]:
constructions = find_relevant_constructions('NP')
constructions[-20:]

[(NP -> PRP, 280),
 (NP -> NNP NNP NNP, 282),
 (NP -> NP CC NP, 289),
 (NP -> NN NNS, 304),
 (NP -> DT NN NN, 313),
 (NP -> CD, 327),
 (NP -> DT NNS, 358),
 (NP -> NP PP-LOC, 363),
 (NP -> QP -NONE-, 365),
 (NP -> JJ NN, 390),
 (NP -> NP SBAR, 409),
 (NP -> JJ NNS, 653),
 (NP -> NNP NNP, 734),
 (NP -> DT JJ NN, 740),
 (NP -> NNP, 837),
 (NP -> NNS, 996),
 (NP -> NN, 1110),
 (NP -> -NONE-, 1225),
 (NP -> DT NN, 2020),
 (NP -> NP PP, 2188)]

In [10]:
np_np_pp_frq = 2188 + 363
np_n_without_frq = 2020 + 358

np_with_pp = np_np_pp_frq / (np_np_pp_frq + np_n_without_frq)
np_without = np_n_without_frq / (np_np_pp_frq + np_n_without_frq)

(np_with_pp, np_without)

(0.5175491986204098, 0.4824508013795902)

---
### 2.3: Regelwahrscheinlichkeiten für DET-Regeln abschätzen


#### Zählen Sie auch für die `DET`-Erweiterungen nach *a* bzw. *the*, wie oft sie in der Penn Treebank (`DT`) vorkommen und berechnen Sie die relativen Häufigkeiten als Approximation der Regelwahrscheinlichkeiten:

$$P(the \mid DET) = \dfrac{count(DET \rightarrow the)}{count(V\!P \rightarrow \setminus*)}$$

**hier:**

$$= \dfrac{count(DET \rightarrow the)}{count(DET \rightarrow the) + count(DET \rightarrow a/an)}$$

In [11]:
constructions = find_relevant_constructions('DT')
constructions[-30:]

[(DT -> 'Those', 6),
 (DT -> 'Another', 6),
 (DT -> 'neither', 7),
 (DT -> 'Both', 9),
 (DT -> 'Each', 9),
 (DT -> 'No', 10),
 (DT -> 'half', 10),
 (DT -> 'All', 12),
 (DT -> 'An', 18),
 (DT -> 'every', 19),
 (DT -> 'These', 22),
 (DT -> 'Some', 22),
 (DT -> 'both', 34),
 (DT -> 'That', 37),
 (DT -> 'each', 37),
 (DT -> 'This', 40),
 (DT -> 'another', 42),
 (DT -> 'those', 55),
 (DT -> 'these', 55),
 (DT -> 'no', 76),
 (DT -> 'that', 77),
 (DT -> 'all', 86),
 (DT -> 'any', 103),
 (DT -> 'A', 105),
 (DT -> 'some', 122),
 (DT -> 'this', 184),
 (DT -> 'an', 316),
 (DT -> 'The', 713),
 (DT -> 'a', 1874),
 (DT -> 'the', 4038)]

In [12]:
det_the_frq = 4038 + 713
det_a_frq = 1874 + 316 + 105 + 18

det_the = det_the_frq / (det_the_frq + det_a_frq)
det_a = det_a_frq / (det_the_frq + det_a_frq)

(det_the, det_a)

(0.6725651189127972, 0.3274348810872027)

---
### 2.4: Erstellen einer PCFG

#### Die aus den Daten extrahierten relativen Häufigkeiten sollen nun zur Erstellung einer probabilistischen kontextfreien Grammatik (PCFG)  genutzt werden.

In [13]:
(vp_with_pp, vp_without, np_with_pp, np_without, det_the, det_a)

(0.12585895117540688,
 0.8741410488245931,
 0.5175491986204098,
 0.4824508013795902,
 0.6725651189127972,
 0.3274348810872027)

In [14]:
pcfg = """
S -> NP VP     [1.0]
VP -> V NP PP  [{}]
VP -> V NP     [{}]
NP -> DET N    [{}]
NP -> NP PP    [{}]
PP -> P NP     [1.0]

DET -> "the"     [{}]
DET -> "a"       [{}]
N -> "boy"       [0.4]
N -> "woman"     [0.4]
N -> "telescope" [0.2]
V -> "saw"       [1.0]
P -> "with"      [1.0]
""".format(
    vp_with_pp, vp_without, np_without,
    np_with_pp, det_the, det_a
)
grammar = nltk.PCFG.fromstring(pcfg)
print(grammar)

Grammar with 13 productions (start state = S)
    S -> NP VP [1.0]
    VP -> V NP PP [0.125859]
    VP -> V NP [0.874141]
    NP -> DET N [0.482451]
    NP -> NP PP [0.517549]
    PP -> P NP [1.0]
    DET -> 'the' [0.672565]
    DET -> 'a' [0.327435]
    N -> 'boy' [0.4]
    N -> 'woman' [0.4]
    N -> 'telescope' [0.2]
    V -> 'saw' [1.0]
    P -> 'with' [1.0]


---
### 2.5: Verwendung zur Disambiguierung

#### Testen Sie Ihre so erstellte Grammatik nun, indem Sie folgenden Satz parsen:

- *the boy saw a woman with a telescope*

In [15]:
parser = nltk.ViterbiParser(grammar)
for tree in parser.parse("the boy saw a woman with a telescope".split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP (DET the) (N boy))
  (VP
    (V saw)
    (NP
      (NP (DET a) (N woman))
      (PP (P with) (NP (DET a) (N telescope)))))) (p=0.000117227)
                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌────────────┴────┐                     
     │       │                 NP                   
     │       │       ┌─────────┴────┐                
     │       │       │              PP              
     │       │       │         ┌────┴───┐            
     NP      │       NP        │        NP          
 ┌───┴───┐   │   ┌───┴────┐    │    ┌───┴──────┐     
DET      N   V  DET       N    P   DET         N    
 │       │   │   │        │    │    │          │     
the     boy saw  a      woman with  a      telescope



---

###  2.6 Wenn Sie sich die extrahierten Wahrscheinlichkeiten und das disambiguierte Ergebnis ansehen, überrascht Sie dann das Ergebnis der Syntaxanalyse?

In [16]:
# relevante Teilwahrscheinlichkeit VP-Attachment:
vp_with_pp * np_without
#P(VP->VP+NP+PP) * P(NP->DET+N)

0.060720751855369764

In [17]:
# relevante Teilwahrscheinlichkeit NP-Attachment (s.u.):
vp_without * np_with_pp * np_without
#P(VP->VP+NP) * P(NP->NP+PP) * P(NP->DET+N)

0.218266049165406

---

### 2.7 Vergleichen Sie dieses Ergebnis mit der PCFG-Analyse mit folgenden abweichenden Regelwahrscheinlichkeiten. 

### Warum wird hier trotz `vp_with_pp < np_with_pp` der VP-Attachment-Baum als der wahrscheinlichere ausgewählt? 

##### (Beachten Sie die Anzahl an Regelanwendungen in den beiden Syntaxbäumen!)


In [18]:
vp_with_pp = 0.2
vp_without = 0.8
np_with_pp = 0.22
np_without = 0.78

In [19]:
pcfg = """
S -> NP VP     [1.0]
VP -> V NP PP  [{}]
VP -> V NP     [{}]
NP -> DET N    [{}]
NP -> NP PP    [{}]
PP -> P NP     [1.0]

DET -> "the"     [{}]
DET -> "a"       [{}]
N -> "boy"       [0.4]
N -> "woman"     [0.4]
N -> "telescope" [0.2]
V -> "saw"       [1.0]
P -> "with"      [1.0]
""".format(
    vp_with_pp, vp_without, np_without,
    np_with_pp, det_the, det_a
)

grammar = nltk.PCFG.fromstring(pcfg)
print(grammar)


Grammar with 13 productions (start state = S)
    S -> NP VP [1.0]
    VP -> V NP PP [0.2]
    VP -> V NP [0.8]
    NP -> DET N [0.78]
    NP -> NP PP [0.22]
    PP -> P NP [1.0]
    DET -> 'the' [0.672565]
    DET -> 'a' [0.327435]
    N -> 'boy' [0.4]
    N -> 'woman' [0.4]
    N -> 'telescope' [0.2]
    V -> 'saw' [1.0]
    P -> 'with' [1.0]


In [20]:
parser = nltk.ViterbiParser(grammar)
for tree in parser.parse("the boy saw a woman with a telescope".split()):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP (DET the) (N boy))
  (VP
    (V saw)
    (NP (DET a) (N woman))
    (PP (P with) (NP (DET a) (N telescope))))) (p=0.000219002)
                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌───────┬────┴─────────┐                
     │       │       │              PP              
     │       │       │         ┌────┴───┐            
     NP      │       NP        │        NP          
 ┌───┴───┐   │   ┌───┴────┐    │    ┌───┴──────┐     
DET      N   V  DET       N    P   DET         N    
 │       │   │   │        │    │    │          │     
the     boy saw  a      woman with  a      telescope



In [21]:
# relevante Teilwahrscheinlichkeit VP-Attachment:
vp_with_pp * np_without
#P(VP->VP+NP+PP) * P(NP->DET+N)

0.15600000000000003

In [22]:
# relevante Teilwahrscheinlichkeit NP-Attachment (1 Regel mehr im Baum, wegen rekursiver Regel NP->NP+PP):
vp_without * np_with_pp * np_without
#P(VP->VP+NP) * P(NP->NP+PP) * P(NP->DET+N)

0.13728

---
#### vgl. die Anzahl der Regeln in den beiden Bäumen:

In [23]:
sentence = "the boy saw a woman with a telescope"
parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)
    for i, prod in enumerate(tree.productions()):
        print(i, prod)

                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌───────┬────┴─────────┐                
     │       │       │              PP              
     │       │       │         ┌────┴───┐            
     NP      │       NP        │        NP          
 ┌───┴───┐   │   ┌───┴────┐    │    ┌───┴──────┐     
DET      N   V  DET       N    P   DET         N    
 │       │   │   │        │    │    │          │     
the     boy saw  a      woman with  a      telescope

0 S -> NP VP
1 NP -> DET N
2 DET -> 'the'
3 N -> 'boy'
4 VP -> V NP PP
5 V -> 'saw'
6 NP -> DET N
7 DET -> 'a'
8 N -> 'woman'
9 PP -> P NP
10 P -> 'with'
11 NP -> DET N
12 DET -> 'a'
13 N -> 'telescope'
                 S                                  
     ┌───────────┴────────┐                          
     │                    VP                        
     │       ┌────────────┴────┐                